### consumindo dados da api 

In [ ]:
import requests
import pandas as pd

def get_swapi_data(endpoint):
    """Função para consumir a API do Star Wars."""
    url = f"https://swapi.dev/api/{endpoint}"
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.json()
    return None  # Retorna None caso o status não seja 200

# Dicionário para armazenar os dados de cada categoria
swapi_data = {
    "people": [],
    "planets": [],
    "films": [],
    "species": [],
    "vehicles": [],
    "starships": []
}

# Função para coletar os dados até encontrar 10 erros 404 consecutivos
def collect_data(category):
    id_ = 1  # Começamos do ID 1
    not_found_count = 0  # Contador de erros 404 consecutivos

    while not_found_count < 10:  # Encerra apenas após 10 erros 404 seguidos
        data = get_swapi_data(f"{category}/{id_}")

        if data is None:
            not_found_count += 1  # Incrementa contador de 404 consecutivos
            print(f"{category.capitalize()} ID {id_} não encontrado. Contagem de 404: {not_found_count}")
        else:
            not_found_count = 0  # Reseta o contador se encontrar um ID válido
            swapi_data[category].append(data)

        id_ += 1  # Avança para o próximo ID
    
    print(f"Fim da coleta para {category}! Total coletado: {len(swapi_data[category])}")

# Coletar dados para cada categoria
categories = ["people", "planets", "films", "species", "vehicles", "starships"]

for category in categories:
    collect_data(category)

# Convertendo para DataFrames para facilitar a análise
df_people = pd.DataFrame(swapi_data["people"])
df_planets = pd.DataFrame(swapi_data["planets"])
df_films = pd.DataFrame(swapi_data["films"])
df_species = pd.DataFrame(swapi_data["species"])
df_vehicles = pd.DataFrame(swapi_data["vehicles"])
df_starships = pd.DataFrame(swapi_data["starships"])


### conferindo dataframes 

In [ ]:
df_people
# df_planets
# df_films
# df_species
# df_vehicles
# df_starships

### transformando dados, agragando dados interessantes em um DF só.

In [ ]:
import pandas as pd
import ast

# Carregar os DataFrames
df_people_enriched = df_people

# Função para substituir URLs por nomes
def substituir_urls(df_base, coluna, df_mapeamento, chave, valor):
    mapeamento = dict(zip(df_mapeamento[chave], df_mapeamento[valor]))
    df_base[coluna] = df_base[coluna].apply(lambda x: ', '.join([mapeamento.get(url, url) for url in ast.literal_eval(x)]) if isinstance(x, str) else x)

# Substituir homeworld (planeta)
df_people_enriched['homeworld'] = df_people_enriched['homeworld'].map(dict(zip(df_planets['url'], df_planets['name'])))

# Substituir films (filmes)
substituir_urls(df_people_enriched, 'films', df_films, 'url', 'title')

# Substituir species (espécies)
substituir_urls(df_people_enriched, 'species', df_species, 'url', 'name')

# Substituir vehicles (veículos)
substituir_urls(df_people_enriched, 'vehicles', df_vehicles, 'url', 'name')

# Substituir starships (naves)
substituir_urls(df_people_enriched, 'starships', df_starships, 'url', 'name')

# Criar mapeamento de personagens por filme
film_characters_map = {row['url']: ast.literal_eval(row['characters']) for _, row in df_films.iterrows()}

# Criar dicionário para mapear URLs de personagens para seus nomes
character_name_map = dict(zip(df_people_enriched['url'], df_people_enriched['name']))

# Criar nova coluna com os personagens que apareceram nos mesmos filmes
def obter_colegas(films, self_url):
    if isinstance(films, str):
        films_list = films.split(', ')  # Já está no formato de nomes de filmes
        personagens = set()
        for film in df_films[df_films['title'].isin(films_list)]['characters']:
            personagens.update(ast.literal_eval(film))
        personagens.discard(self_url)  # Remover o próprio personagem
        return ', '.join([character_name_map.get(url, url) for url in personagens])
    return ""

df_people_enriched['characters_met'] = df_people_enriched.apply(lambda row: obter_colegas(row['films'], row['url']), axis=1)

# Salvar o resultado final
df_people_enriched.to_csv('people_enriquecido.csv', index=False)


In [ ]:
df_people_enriched.columns
df_people_enriched.to_json('star_wars_characters.json')